# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=120                             #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario_2"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv_sharedPolicy_4veh"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW', 'TenaciousDScenario', 'IntersectionTWScenario_2']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3', 'MultiAgentIntersectionEnv_sharedPolicy_TeamSpirit', 'MultiTenaciousDEnv', 'MultiAgentIntersectionEnv_sharedPolicy_2veh', 'MultiAgentIntersectionEnv_sharedPolicy_4veh']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 80,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=0,                               #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=4,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 200

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-02_10-37-15_29615/logs.
Waiting for redis server at 127.0.0.1:45900 to respond...
Waiting for redis server at 127.0.0.1:56386 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8891/notebooks/ray_ui.ipynb?token=531b4b319102dd18351d8d87b240733bacbc0a59861293a8



{'node_ip_address': '172.16.123.117',
 'object_store_addresses': ['/tmp/ray/session_2019-05-02_10-37-15_29615/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-05-02_10-37-15_29615/sockets/raylet'],
 'redis_address': '172.16.123.117:45900',
 'webui_url': 'http://localhost:8891/notebooks/ray_ui.ipynb?token=531b4b319102dd18351d8d87b240733bacbc0a59861293a8'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50, 25]})  # size of hidden layers in network defaule 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_paramshttps://www.tourdatenarchiv.de/setlist/71/07/Es-wird-eng/Frankfurt-Main-Festhalle/
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return 'rl_0'

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn),
            'policies_to_train': ['rl_0']
        }
    })

 Starting SUMO on port 49175


New Teamspirit:
0.04880195980933988
-0.5698350356183577
[('bottom_intersection', 33.435266406731216), ('bottom_intersection', 49.43253509925844), ('top_intersection', 13.348699041252347), ('top_intersection', 52.85898909213359)]


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0_2019-05-02_10-37-18hbo7ufcc -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.9/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_10-39-22
  done: false
  episode_len_mean: 119.25
  episode_reward_max: 44.71218454332695
  episode_reward_mean: 18.806414594426464
  episode_reward_min: -389.4910954789854
  episodes_this_iter: 200
  episodes_total: 200
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 20570.786
    load_time_ms: 4

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:	RUNNING [pid=29660], 470 s, 5 iter, 120000 ts, -62.8 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_10-47-05
  done: false
  episode_len_mean: 106.5911111111111
  episode_reward_max: 210.9356255193253
  episode_reward_mean: -46.558701744159364
  episode_reward_min: -350.2312174305498
  episodes_this_iter: 225
  episodes_total: 1258
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 20545.255
    load_time_ms: 11.05
    num_steps_sampled: 144000
    num_steps_trained: 144000
    rl_0:
      cur_kl_coeff: 0.01250000111758709
      cur_lr: 4.999999873689376e-05
      entropy: 1.3857861757278442
      kl: 0.0058530461974442005
 

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_10-56-40
  done: false
  episode_len_mean: 95.95617529880478
  episode_reward_max: 306.7063225335668
  episode_reward_mean: -59.62094419283321
  episode_reward_min: -357.6593962816094
  episodes_this_iter: 251
  episodes_total: 2464
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 23900.515
    load_time_ms: 4.205
    num_steps_sampled: 264000
    num_steps_trained: 264000
    rl_0:
      cur_kl_coeff: 0.00039062503492459655
      cur_lr: 4.999999873689376e-05
      entropy: 1.352575659751892
      kl: 0.00715090986341238
      policy_loss: -0.0019051320850849152
      total_loss: 589.0093994140625
      vf_explained_var: 0.5111197829246521
      vf_loss: 589.01123046875
    sample_time_ms: 79718.244
    update_time_ms: 6.567
  iterations_since_restore: 11
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 29660
  policy_r

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 4.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:	RUNNING [pid=29660], 1660 s, 16 iter, 384000 ts, -11.8 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_11-06-54
  done: false
  episode_len_mean: 90.2593984962406
  episode_reward_max: 375.74816309415246
  episode_reward_mean: -45.605702323018804
  episode_reward_min: -362.0269074270523
  episodes_this_iter: 266
  episodes_total: 4013
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 23838.648
    load_time_ms: 4.084
    num_steps_sampled: 408000
    num_steps_trained: 408000
    rl_0:
      cur_kl_coeff: 6.103516170696821e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.314544439315796
      kl: 0.004707448184490204

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_11-14-24
  done: false
  episode_len_mean: 90.3796992481203
  episode_reward_max: 443.45326225471337
  episode_reward_mean: 17.117335466812612
  episode_reward_min: -358.6292672963119
  episodes_this_iter: 266
  episodes_total: 5340
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 19337.344
    load_time_ms: 3.914
    num_steps_sampled: 528000
    num_steps_trained: 528000
    rl_0:
      cur_kl_coeff: 1.9073488033427566e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.2755107879638672
      kl: 0.005140467546880245
      policy_loss: -0.0012375077931210399
      total_loss: 539.9653930664062
      vf_explained_var: 0.7504706382751465
      vf_loss: 539.9666137695312
    sample_time_ms: 74500.491
    update_time_ms: 6.277
  iterations_since_restore: 22
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 29660
  poli

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 7.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:	RUNNING [pid=29660], 15275 s, 159 iter, 3816000 ts, 179 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_14-53-54
  done: false
  episode_len_mean: 90.56439393939394
  episode_reward_max: 634.9628674536231
  episode_reward_mean: 245.9334480092195
  episode_reward_min: -352.9140692205035
  episodes_this_iter: 264
  episodes_total: 42850
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 19867.523
    load_time_ms: 3.791
    num_steps_sampled: 3840000
    num_steps_trained: 3840000
    rl_0:
      cur_kl_coeff: 6.018531614308722e-37
      cur_lr: 4.999999873689376e-05
      entropy: 0.2623051404953003
      kl: 0.012673323042690

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_15-01-22
  done: false
  episode_len_mean: 86.00358422939068
  episode_reward_max: 633.3031617660454
  episode_reward_mean: 186.86793519539907
  episode_reward_min: -356.14287738855904
  episodes_this_iter: 279
  episodes_total: 44243
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 18889.033
    load_time_ms: 3.677
    num_steps_sampled: 3960000
    num_steps_trained: 3960000
    rl_0:
      cur_kl_coeff: 6.018531614308722e-37
      cur_lr: 4.999999873689376e-05
      entropy: 0.21115796267986298
      kl: 0.012431297451257706
      policy_loss: 0.00016732708900235593
      total_loss: 610.74609375
      vf_explained_var: 0.8934885859489441
      vf_loss: 610.7459106445312
    sample_time_ms: 70660.017
    update_time_ms: 5.355
  iterations_since_restore: 165
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 29660
  poli

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 8.2/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:	RUNNING [pid=29660], 16387 s, 170 iter, 4080000 ts, 213 rew

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_15-13-02
  done: false
  episode_len_mean: 86.52688172043011
  episode_reward_max: 640.691631062095
  episode_reward_mean: 199.4859566112759
  episode_reward_min: -355.1839836238594
  episodes_this_iter: 279
  episodes_total: 45891
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 23198.541
    load_time_ms: 4.099
    num_steps_sampled: 4104000
    num_steps_trained: 4104000
    rl_0:
      cur_kl_coeff: 6.018531614308722e-37
      cur_lr: 4.999999873689376e-05
      entropy: 0.16033411026000977
      kl: 0.015169168822467

Result for PPO_MultiAgentIntersectionEnv_sharedPolicy_4veh-v0_0:
  custom_metrics: {}
  date: 2019-05-02_15-24-15
  done: false
  episode_len_mean: 87.78832116788321
  episode_reward_max: 641.5453994260478
  episode_reward_mean: 206.55034429643183
  episode_reward_min: -353.6894277979851
  episodes_this_iter: 274
  episodes_total: 47256
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 28740.947
    load_time_ms: 4.754
    num_steps_sampled: 4224000
    num_steps_trained: 4224000
    rl_0:
      cur_kl_coeff: 6.018531614308722e-37
      cur_lr: 4.999999873689376e-05
      entropy: 0.12763839960098267
      kl: 0.013762842863798141
      policy_loss: 0.0004640940169338137
      total_loss: 612.9729614257812
      vf_explained_var: 0.8857985138893127
      vf_loss: 612.9724731445312
    sample_time_ms: 98764.702
    update_time_ms: 10.037
  iterations_since_restore: 176
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 29660
  

  custom_metrics: {}
  date: 2019-05-02_15-33-34
  done: false
  episode_len_mean: 89.46468401486989
  episode_reward_max: 642.8047096518987
  episode_reward_mean: 240.57656092214492
  episode_reward_min: -349.508659526333
  episodes_this_iter: 269
  episodes_total: 48639
  experiment_id: 617513c78ba74eb29654585d798eef12
  hostname: Gandalf
  info:
    grad_time_ms: 28013.175
    load_time_ms: 4.546
    num_steps_sampled: 4344000
    num_steps_trained: 4344000
    rl_0:
      cur_kl_coeff: 6.018531614308722e-37
      cur_lr: 4.999999873689376e-05
      entropy: 0.10201722383499146
      kl: 0.014210670255124569
      policy_loss: 0.001091307494789362
      total_loss: 740.3943481445312
      vf_explained_var: 0.8542518019676208
      vf_loss: 740.3931884765625
    sample_time_ms: 95004.589
    update_time_ms: 11.693
  iterations_since_restore: 181
  node_ip: 172.16.123.117
  num_metric_batches_dropped: 0
  pid: 29660
  policy_reward_mean:
    rl_0: 60.144140230536244
  time_since_resto